In [ ]:
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render

import matplotlib.pyplot as plt

import pickle

# Load results 
* BO without any constraint --> `results/baseline_1601.json`
* BO with OC --> `results/outconst_1601.json`
* BO with ReturnInf --> `results/returninf_1601.json`
* BO with ReturnInf and OC --> `results/returninf_1601_with_oc.json`
* BO with LinearComb --> `results/lin_comb_1601.json` and `results/lin_comb_1601_second_run.json`
* BO with LinearComb and OC --> ___


* Random search --> `results/random_search_results_1701.pickle`
* Grid search --> `results/grid_search_results_1601.pickle`

In [ ]:
bo_no_constraint = AxClient.load_from_json_file("results/baseline_1601.json")

In [ ]:
bo_oc = AxClient.load_from_json_file("results/outconst_1601.json")

In [ ]:
bo_ri = AxClient.load_from_json_file("results/returninf_1601.json")

In [ ]:
bo_ri_oc = AxClient.load_from_json_file("results/returninf_1601_with_oc.json")

In [ ]:
bo_lc = AxClient.load_from_json_file("results/lin_comb_1601.json")

In [ ]:
bo_lc_second = AxClient.load_from_json_file("results/lin_comb_1601_second_run.json")

In [ ]:
with open('results/random_search_results_1701.pickle', 'rb') as file:
    random_search = pickle.load(file)

In [ ]:
with open('results/grid_search_results_1601.pickle', 'rb') as file:
    grid_search = pickle.load(file)

# Plot convergence 

In [ ]:
BASELINE_SIZE = 78172

def get_convergence(ax_client, lin_comb=False): 
    samples = ax_client.get_trials_data_frame().values
    
    min_sizes = [] 
    current_min_size = BASELINE_SIZE
    
    for sample in samples: 
        acc = sample[0]
        if not lin_comb: 
            size = sample[1]
        else: 
            size = sample[2]
        
        if acc < 0.95: 
            min_sizes.append(current_min_size)
        else: 
            current_min_size = min(current_min_size, size)
            min_sizes.append(current_min_size)
    return min_sizes

In [ ]:
def plot_convergences(ax_clients, lin_comb=False): 
    plt.figure(figsize=(15,6))
    for (name, ax_client) in ax_clients: 
        plt.plot(get_convergence(ax_client, lin_comb), label=name)


In [ ]:
def plot_convergence(ax_client, lin_comb=False): 

    plt.figure(figsize=(15,6))
    plt.plot(get_convergence(ax_client, lin_comb))
    plt.show()

# Convergence graph

In [ ]:
random_bests = []
current_best = random_search[0][1]["size"][0]
params_bests = []
params_current_best = random_search[0][0]
acc_bests = [] 
acc_current_best = random_search[0][1]["accuracy"][0]

for par, metrics in random_search: 
    
    acc = metrics["accuracy"][0]
    size = metrics["size"][0]
    
    if acc < 0.95: 
        params_bests.append(params_current_best)
        acc_bests.append(acc_current_best)
        random_bests.append(current_best)
    elif size < current_best: 
        params_current_best = par
        params_bests.append(par) 
        acc_current_best = acc
        acc_bests.append(acc)
        current_best = size
        random_bests.append(size)
    else: 
        params_bests.append(params_current_best) 
        acc_bests.append(acc_current_best)
        random_bests.append(current_best)

In [ ]:
GRID_MIN = min(filter(lambda x: x[1]["accuracy"][0] >= 0.95, grid_search), key=lambda x: x[1]["size"][0])

In [ ]:
plt.rcParams["font.size"] = "13"

In [ ]:
plt.figure(figsize=(15,6))
# plt.plot(get_convergence(bo_no_constraint), label="Baseline BO", linewidth=2)
plt.plot(get_convergence(bo_oc), label="BO with Outcome Constraint", linewidth=2)
plt.plot(get_convergence(bo_ri_oc), label="BO with Return Baseline", linewidth=2)
plt.plot(get_convergence(bo_lc_second, True), label="BO with Linear Combination", linewidth=2)
plt.plot(random_bests[:50], label="Random Search", linewidth=2)
plt.axhline(GRID_MIN[1]["size"][0], linestyle="--", color="black", 
            label="Best value found by Grid Search", linewidth=2)


plt.ylim((5500,10000))
plt.xlabel("Number of iterations")
plt.ylabel("Best valid model size (bytes)")
# plt.yscale('log') 
plt.legend()
plt.savefig("graphs/convergence.svg")
plt.show()

In [ ]:
plot_convergences([("no constraint", bo_no_constraint), ("ret inf", bo_ri)])
plt.plot(get_convergence(bo_lc_second))

In [ ]:
plot_convergence(bo_no_constraint)

In [ ]:
plot_convergence(bo_oc)

In [ ]:
plot_convergence(bo_ri)

In [ ]:
plot_convergence(bo_ri_oc)

In [ ]:
plot_convergence(bo_lc, True)

In [ ]:
plot_convergence(bo_lc_second, True)

In [ ]:
plt.plot(get_convergence(bo_lc))
plt.plot(get_convergence(bo_lc_second))

In [ ]:
bo_lc_second.get_best_parameters()

In [ ]:
bo_lc_second.get_trials_data_frame().

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(x, get_convergence(bo_oc))
x = range(1, len(get_convergence(bo_oc)) + 1)
plt.errorbar(x, get_convergence(bo_oc), get_size_sem(bo_oc))

In [ ]:
bo_oc.get_optimization_trace()

In [ ]:
bo_oc.experiment.trials[1]

In [ ]:
bo_oc.get_trials_data_frame()

In [ ]:
def get_size_sem(ax_client): 
    # Note: get only the 'size' metric -> hence, [::2]
    samples = ax_client.experiment.fetch_data().df.values[::2]
    return [x[3] for x in samples]

In [ ]:
def get_size_means(ax_client): 
    # Note: get only the 'size' metric -> hence, [::2]
    samples = ax_client.experiment.fetch_data().df.values[::2]
    return [x[3] for x in samples]

In [ ]:
bo_oc.experiment.fetch_data().df.values[::2][:3]

In [ ]:
bo_oc.experiment.fetch_data().df.values[1::2][:3]

In [ ]:
def get_metrics(ax_client): 
    sizes = ax_client.experiment.fetch_data().df.values[::2]
    accuracies = ax_client.experiment.fetch_data().df.values[1::2]
    
    return list(zip([x[2] for x in sizes], [x[3] for x in sizes], [x[2] for x in accuracies]))

In [ ]:
metrics = get_metrics(bo_oc)

In [ ]:
metrics[0]

In [ ]:
plot_vals = []
plot_errs = [] 

current_best = metrics[0][0]
current_err = metrics[0][1]

for (size, err, acc) in metrics: 
    if acc >= 0.95 and size < current_best:
        current_best = size 
        current_err = err 
    plot_vals.append(current_best)
    plot_errs.append(current_err)


In [ ]:
xs = list(range(len(plot_vals)))

plt.figure(figsize=(10,10))
plt.plot(xs, plot_vals)
plt.errorbar(xs, plot_vals, yerr=plot_errs)

In [ ]:
list(filter(lambda x: x[2] > 0.95, metrics))

In [ ]:
def plot_samples(ax_client):
    res = ax_client.get_trials_data_frame()
    acc = res["accuracy"]
    siz = res["size"]
    
    plt.scatter(acc, siz)
    plt.axvline(x=0.95, color="red")
    plt.xlim(0, 1)
    plt.ylim(0, 15000)
    
#     fig, ax = plt.subplots()
#     ax.scatter(acc, siz)
    
#     n = list(range(1,len(siz) + 1))

#     for i, txt in enumerate(n):
#         ax.annotate(txt, (acc[i], siz[i]))

#     ax.axvline(x=0.95)

In [ ]:
plot_samples(bo_no_constraint)

In [ ]:
plot_samples(bo_oc)

In [ ]:
plot_samples(bo_ri)

In [ ]:
plot_samples(bo_ri_oc)

In [ ]:
plot_samples(bo_lc)

In [ ]:
plot_samples(bo_lc_second)

In [ ]:
random_sizs = [x[1]["size"][0] for x in random_search]
random_accs = [x[1]["accuracy"][0] for x in random_search]